In [1]:
import os
import sys
from pathlib import Path

repo_dir = Path.cwd().parents[0]
if repo_dir.as_posix() not in sys.path:
    sys.path.append(repo_dir.as_posix())

In [2]:
from src.store import vectorize_faqs_from_csv

In [3]:
csv_dir = repo_dir.joinpath("data", "summarized_texts")  # フォーマット済みテキストファイルのディレクトリ
vector_store_dir = repo_dir.joinpath("data", "vectorize_dir", "faq_csv_1")        # ベクトルストアの保存先ディレクトリ
collection_name = "faq_csv_1"                # Chromaコレクション名

retriever = vectorize_faqs_from_csv(
    csv_dir=str(csv_dir),
    vector_store_dir=str(vector_store_dir),
    collection_name=str(collection_name)
)

/home/atsushi/repositories/RAG-FDUA/src/store/vectorize.py:124: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small")
2025-01-20 06:43:33,419 INFO:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2025-01-20 06:43:35,558 INFO:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
/home/atsushi/repositories/RAG-FDUA/src/store/vectorize.py:87: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()
2025-01-20 06

In [12]:
import os
import sys
import time
from pathlib import Path
from dotenv import load_dotenv
from typing import List, Dict
import pandas as pd
import tiktoken

from langchain.docstore.document import Document
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
import logging  # ロギングのインポート

def retrieve(state: dict, retriever, top_k: int = 5) -> dict:
    """
    Retrieve documents based on the question in the state.

    Args:
        state (dict): The current graph state containing the question.
        retriever: The retriever object for vectorized document search.
        top_k (int): Number of top documents to retrieve.

    Returns:
        dict: Updated state with retrieved documents.
    """
    print("---RETRIEVE---")
    
    # Ensure the question exists in the state
    question = state.get("question")
    if not question:
        raise ValueError("The state must contain a 'question' key with a valid question string.")

    try:
        # Retrieve relevant documents using the retriever
        logging.info(f"[INFO] Retrieving top {top_k} documents for the question: {question}")
        results = retriever.get_relevant_documents(question)
        
        # Format the retrieved documents for the state
        retrieved_docs = [
            {
                "content": doc.page_content,
                "metadata": doc.metadata
            }
            for doc in results[:top_k]
        ]

        # Update the state with the retrieved documents
        state["retrieved_documents"] = retrieved_docs
        logging.info(f"[INFO] Retrieved {len(retrieved_docs)} documents for the question.")
        return state  # 明確に state を返す

    except Exception as e:
        logging.error(f"[ERROR] Failed to retrieve documents: {e}")
        state["retrieved_documents"] = []
        return state  # エラー時にも state を返す



In [14]:
# 仮の状態を用意
state = {
    "question": "IHIグループの経営目標の進捗の2022年度と2023年度を比較したとき、営業利益率と税引後ROICではどちらの方が差が大きいか"
}

vector_store_dir = str(repo_dir.joinpath("data", "vectorize_dir", "faq_csv_1"))        # ベクトルストアの保存先ディレクトリ
collection_name = "faq_csv_1"   

retriever = Chroma(
    persist_directory=vector_store_dir,
    collection_name=collection_name,
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv("OPENAI_API_KEY"))
).as_retriever()

# 関数を呼び出し
updated_state = retrieve(state, retriever)

# 結果の確認
print(updated_state["retrieved_documents"])

2025-01-20 07:21:55,319 INFO:[INFO] Retrieving top 5 documents for the question: IHIグループの経営目標の進捗の2022年度と2023年度を比較したとき、営業利益率と税引後ROICではどちらの方が差が大きいか


---RETRIEVE---


2025-01-20 07:21:55,836 INFO:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-01-20 07:21:55,845 INFO:[INFO] Retrieved 4 documents for the question.


[{'content': '質問: 日本の家電商品の修理サービスはどの会社が担当していますか？\n回答: 日本の家電商品の修理サービスは、パナソニック マーケティングジャパン（株）内のCS社（修理・部品サービス部門）が担当しています。', 'metadata': {'file_name': '10', 'page_number': 119}}, {'content': '質問: 取締役および監査役の候補者はどのように決定されますか？\n回答: 取締役および監査役の候補者は、指名・報酬委員会の協議を経て取締役会で決定され、株主総会に提案する際には選定理由が説明されます。', 'metadata': {'file_name': '15', 'page_number': 33}}, {'content': '質問: パナソニック株式会社の社員の健康支援に関する具体的な取り組みは何ですか？\n回答: パナソニック株式会社では、産業医・産業保健スタッフが常駐する健康管理室を設置し、生活習慣病予防や禁煙等の健康支援を行っています。', 'metadata': {'file_name': '10', 'page_number': 104}}, {'content': '質問: パナソニックグループが中国での廃旧家電回収目標責任制度にどのように対応しているか？\n回答: パナソニックグループは、2017年1月に政府より公布された生産者責任延伸制度推進方案や、2020年9月に施行された固体廃棄物環境汚染防止法などの関連政策に注目し、対応検討を進めています。\n\n質問: パナソニックグループがサプライチェーン連携において重視している環境課題は何ですか？\n回答: パナソニックグループは、製品や部品の再利用の強化、製品のリサイクル性を改善する方法について、グループ内での議論をさらに強化しています。', 'metadata': {'file_name': '10', 'page_number': 54}}]
